In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from statsmodels import robust
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import os
import matplotlib.cm as cm

# run pca and visualize
from mpl_toolkits.mplot3d import Axes3D
from yass.config import Config
from yass.explore.explorers import RecordingExplorer
from pyvenn import do_venn
import scipy

colors = [
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink',
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink',
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink',
'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink',

'black','blue','red','green','cyan','magenta','brown','pink',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink',
    
    
]

sorted_colors=colors

# 49 chans
n_channels = 49
yaml_file_ = '/media/cat/2TB/liam/49channels/data1_allset/ej49_data1_allset.yaml'
geometry_file_49 = '/media/cat/2TB/liam/49channels/data1_allset/ej49_geometry1.txt'
standardized_file_49 = '/media/cat/2TB/liam/49channels/data1_allset/tmp/standarized.bin'

# 512 chans - 2007 dataset
n_channels = 512
yaml_file = '/media/cat/4TB_SSD/liam/512channels/2007/retinal_20min.yaml'
geometry_file_512 = '/media/cat/4TB_SSD/liam/512channels/2007/retinal_20min.txt'
standardized_file_512_2007 = '/media/cat/4TB_SSD/liam/512channels/20mins/tmp/preprocess/standardized.bin'
#CONFIG_512 = Config.from_yaml(yaml_file)


# 512 chans - 2107 dataset
n_channels = 512
yaml_file = '/media/cat/4TB_SSD/liam/512channels/2017-12-04-5/data001/201712-04-5_001.yaml'
geometry_file_512_2017 = '/media/cat/4TB_SSD/liam/512channels/2017-12-04-5/data001/201712-04-5_001.txt'
standardized_file_512_2017  = '/media/cat/4TB_SSD/liam/512channels/2017-12-04-5/data001/tmp/preprocess/standardized.bin'

import torch
from torch import nn
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#import deconv

# visualize recomputed templates over time;
def binary_reader_waveforms(filename, n_channels, n_times, spikes, channels=None, data_type='float32'):
    ''' Reader for loading raw binaries
    
        standardized_filename:  name of file contianing the raw binary
        n_channels:  number of channels in the raw binary recording 
        n_times:  length of waveform 
        spikes: 1D array containing spike times in sample rate of raw data
        channels: load specific channels only
        data_type: float32 for standardized data
    
    '''

    # ***** LOAD RAW RECORDING *****
    if channels is None:
        wfs = np.zeros((spikes.shape[0], n_times, n_channels), data_type)
    else:
        wfs = np.zeros((spikes.shape[0], n_times, channels.shape[0]), data_type)
    if data_type =='float32':
        data_len = 4
    else:
        data_len = 2

    with open(filename, "rb") as fin:
        for ctr,s in enumerate(spikes):
            #print (ctr,s)
            # index into binary file: time steps * 4  4byte floats * n_channels
            fin.seek(s * data_len * n_channels, os.SEEK_SET)
            wfs[ctr] = np.fromfile(
                fin,
                dtype=data_type,
                count=(n_times * n_channels)).reshape(n_times, n_channels)[:,channels]
    fin.close()
    return wfs

Autosaving every 180 seconds


In [2]:
# load raw data and spike train
data = np.fromfile('/media/cat/14TB/liam/512chan/2005-04-26-0/data002/tmp/preprocess/standardized.bin','float32')
print (data.shape)

n_chan = 512
data2D = data.reshape(-1,n_chan)
print (data2D.shape)

spike_train = np.load('/media/cat/14TB/liam/512chan/2005-04-26-0/data002/kilosort2/spike_times_ordered.npy')
print (spike_train)

(3072000000,)
(6000000, 512)
[[    1111        0]
 [    5331        0]
 [    5660        0]
 ...
 [35852953      738]
 [35857218      738]
 [35926027      738]]


In [32]:
# use int16 original binary
if False:
    filename = '/media/cat/12TB/dbox/Dropbox/data_temp/liam/julien/neuropixels/nick_steinmetz/p1_g0_t0.imec0.ap.bin'
    data_type = 'int16'
# use float32 standardized file
else:
    filename = '/media/cat/14TB/liam/512chan/2005-04-26-0/data002/tmp/preprocess/standardized.bin'
    data_type = 'float32'

n_channels = 512
n_times = 101
# spike_times = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_times.npy')
# spike_clusters = np.load('/media/cat/2TB/nick_steinmetz_drift/rec1/KS2/spike_clusters.npy')
#units = np.unique(spike_clusters)
spike_train = np.load('/media/cat/14TB/liam/512chan/2005-04-26-0/data002/kilosort2/spike_times_ordered.npy')

channels = np.arange(n_channels)

#for unit in np.unique(spike_clusters):
#for unit in np.unique(spike_clusters)[:]:
#units = [0]
units = np.unique(spike_train[:,1])
print ("units; ", units.shape)
templates = []
for unit in units:
    if unit%10==0:
        print (unit)
    idx = np.where(spike_train[:,1]==unit)[0]
    spikes = np.int32(spike_train[idx,0])-n_times//2
    # sub sample spikes to speed up loading
    idx = np.where(spikes<5*60*20000-100)[0]
    spikes = spikes[idx]
    
    idx = np.random.choice(np.arange(spikes.shape[0]),min(1000,spikes.shape[0]))
    #print (idx)
    spikes=spikes[idx]
    
    #spikes = spikes[::10]
    wfs = binary_reader_waveforms(filename, n_channels, n_times, spikes, channels, data_type)
    print (unit, wfs.shape)
    if wfs.shape[0]==0:
        templates.append(np.zeros((n_times,n_channels),'float32'))
    else:
        templates.append(wfs.mean(0))
    
    
# max_chan = wfs.mean(0).ptp(0).argmax(0)
# fig = plt.figure()
# plt.plot(wfs[:,:,max_chan].T, c='black',alpha=.02)
# plt.plot(wfs.mean(0)[:,max_chan],c='red')
# plt.show()


units;  (739,)
0
0 (1000, 101, 512)
1 (841, 101, 512)
2 (1000, 101, 512)
3 (1000, 101, 512)
4 (1000, 101, 512)
5 (1000, 101, 512)
6 (1000, 101, 512)
7 (1000, 101, 512)
8 (1000, 101, 512)
9 (1000, 101, 512)
10
10 (1000, 101, 512)
11 (1000, 101, 512)
12 (1000, 101, 512)
13 (1000, 101, 512)
14 (1000, 101, 512)
15 (1000, 101, 512)
16 (1000, 101, 512)
17 (1000, 101, 512)
18 (1000, 101, 512)
19 (1000, 101, 512)
20
20 (1000, 101, 512)
21 (1000, 101, 512)
22 (1000, 101, 512)
23 (1000, 101, 512)
24 (1000, 101, 512)
25 (1000, 101, 512)
26 (1000, 101, 512)
27 (1000, 101, 512)
28 (1000, 101, 512)
29 (1000, 101, 512)
30
30 (1000, 101, 512)
31 (930, 101, 512)
32 (1000, 101, 512)
33 (1000, 101, 512)
34 (1000, 101, 512)
35 (1000, 101, 512)
36 (1000, 101, 512)
37 (104, 101, 512)
38 (1000, 101, 512)
39 (165, 101, 512)
40
40 (1000, 101, 512)
41 (1, 101, 512)
42 (1000, 101, 512)
43 (796, 101, 512)
44 (1000, 101, 512)
45 (1000, 101, 512)
46 (46, 101, 512)
47 (1000, 101, 512)
48 (1000, 101, 512)
49 (1000, 1

395 (1000, 101, 512)
396 (1000, 101, 512)
397 (1000, 101, 512)
398 (1000, 101, 512)
399 (1000, 101, 512)
400
400 (1, 101, 512)
401 (1000, 101, 512)
402 (1000, 101, 512)
403 (1000, 101, 512)
404 (1000, 101, 512)
405 (1000, 101, 512)
406 (1000, 101, 512)
407 (1000, 101, 512)
408 (1000, 101, 512)
409 (1000, 101, 512)
410
410 (1000, 101, 512)
411 (1000, 101, 512)
412 (1000, 101, 512)
413 (1000, 101, 512)
414 (1000, 101, 512)
415 (1000, 101, 512)
416 (1000, 101, 512)
417 (1000, 101, 512)
418 (1000, 101, 512)
419 (978, 101, 512)
420
420 (1000, 101, 512)
421 (1000, 101, 512)
422 (1000, 101, 512)
423 (1000, 101, 512)
424 (674, 101, 512)
425 (1000, 101, 512)
426 (733, 101, 512)
427 (1000, 101, 512)
428 (1000, 101, 512)
429 (1000, 101, 512)
430
430 (1000, 101, 512)
431 (1000, 101, 512)
432 (1000, 101, 512)
433 (1000, 101, 512)
434 (1000, 101, 512)
435 (1000, 101, 512)
436 (1000, 101, 512)
437 (1000, 101, 512)
438 (47, 101, 512)
439 (1000, 101, 512)
440
440 (1000, 101, 512)
441 (1000, 101, 512)
4

In [3]:
# save channels with > 5 templates on them that are > 8.0 SU
temps = np.load('/media/cat/14TB/liam/512chan/2005-04-26-0/data002/kilosort2/templates_reloaded.npy')
print (temps.shape)

ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)

ctr=0
for chan in range(512):
    idx2 = np.where(max_chans==chan)[0]
    ptps_local = ptps[idx2]
    idx3 = np.where(ptps_local>8.0)[0]
    if idx3.shape[0]>5:
        print (chan, idx2.shape, idx3.shape)
                
        # save this channel and the data
        np.save('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/ari_real/2005_kilosort/chan_'+str(ctr)+'_data.npy',
                data2D[:,chan])
        
        # save templates
        np.save('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/ari_real/2005_kilosort/chan_'+str(ctr)+'_templates.npy',
                temps[idx2[idx3],:,chan])
        
        # save spiketrains
        times_out= []
        for id_ in idx2[idx3]:
            idx4 = np.where(spike_train[:,1]==id_)[0]
            times = spike_train[idx4,0]
            idx5 = np.where(times<(5*60*20000))[0]
            times_out.append(times[idx5])
        np.save('/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/ari_real/2005_kilosort/chan_'+str(ctr)+'_times.npy',
                times_out)
            
        ctr+=1

(739, 101, 512)
9 (8,) (8,)
44 (6,) (6,)
113 (6,) (6,)
219 (6,) (6,)
230 (8,) (8,)
309 (7,) (6,)
360 (6,) (6,)
389 (8,) (8,)
395 (8,) (8,)
406 (6,) (6,)
435 (8,) (8,)
453 (6,) (6,)
465 (6,) (6,)
487 (12,) (12,)


In [23]:
# YASS DATASETS
# load raw data and spike train
data = np.fromfile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/preprocess/standardized.bin','float32')
print (data.shape)

n_chan = 512
data2D = data.reshape(-1,n_chan)
print (data2D.shape)

spike_train = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/spike_train.npy')
print (spike_train)

(3072000000,)
(6000000, 512)
[[        0      1430]
 [        0       490]
 [        0       565]
 ...
 [143999995       239]
 [143999996       812]
 [143999997        17]]


In [26]:
# save channels with > 5 templates on them that are > 8.0 SU
out_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/ari/ari_real/2009_yass/'

temps = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/templates/templates_0sec.npy')
print (temps.shape)

ptps = temps.ptp(1).max(1)
max_chans = temps.ptp(1).argmax(1)

ctr=0
for chan in range(512):
    idx2 = np.where(max_chans==chan)[0]
    ptps_local = ptps[idx2]
    idx3 = np.where(ptps_local>8.0)[0]
    if idx3.shape[0]>5:
        print (chan, idx2.shape, idx3.shape)
                
        # save this channel and the data
        np.save(out_dir+'/chan_'+str(ctr)+'_data.npy',
                data2D[:,chan])
        
        # save templates
        np.save(out_dir+'/chan_'+str(ctr)+'_templates.npy',
                temps[idx2[idx3],:,chan])
        
        # save spiketrains
        times_out= []
        for id_ in idx2[idx3]:
            idx4 = np.where(spike_train[:,1]==id_)[0]
            times = spike_train[idx4,0]
            idx5 = np.where(times<(5*60*20000))[0]
            times_out.append(times[idx5])
            
        np.save(out_dir+'/chan_'+str(ctr)+'_times.npy',
                times_out)
        ctr+=1     

(2413, 101, 512)
1 (19,) (17,)
8 (11,) (11,)
29 (8,) (7,)
32 (8,) (7,)
49 (12,) (7,)
53 (25,) (24,)
57 (6,) (6,)
60 (7,) (7,)
64 (24,) (22,)
79 (6,) (6,)
80 (7,) (6,)
94 (10,) (7,)
100 (18,) (6,)
108 (7,) (7,)
117 (6,) (6,)
122 (7,) (7,)
123 (9,) (6,)
128 (8,) (7,)
132 (8,) (6,)
133 (9,) (7,)
142 (8,) (8,)
150 (17,) (16,)
158 (13,) (13,)
169 (9,) (9,)
173 (15,) (14,)
186 (27,) (23,)
188 (33,) (29,)
189 (21,) (15,)
190 (32,) (28,)
191 (9,) (7,)
196 (40,) (29,)
199 (17,) (17,)
201 (20,) (12,)
203 (9,) (8,)
214 (7,) (7,)
215 (35,) (33,)
224 (10,) (6,)
225 (8,) (7,)
226 (11,) (10,)
233 (9,) (9,)
241 (18,) (15,)
246 (6,) (6,)
248 (19,) (13,)
253 (8,) (6,)
270 (13,) (13,)
274 (26,) (25,)
275 (11,) (11,)
278 (8,) (7,)
287 (15,) (8,)
289 (6,) (6,)
290 (18,) (9,)
295 (28,) (13,)
298 (10,) (9,)
300 (10,) (9,)
308 (10,) (8,)
315 (13,) (9,)
317 (9,) (7,)
318 (18,) (17,)
333 (11,) (10,)
335 (27,) (22,)
343 (9,) (7,)
350 (12,) (6,)
353 (8,) (6,)
356 (11,) (11,)
367 (7,) (7,)
369 (15,) (13,)
376 (6,)

In [20]:
data = np.fromfile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/2009-04-13-5.bin','int16',count=512*20000*5*60)
print (data.shape)

(3072000000,)


In [22]:
data.tofile('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/2009-04-13-5_5min.bin')